In [1]:
import os

MODEL_DIR = "agileai_tinyllama_qlora"

print("Files inside model directory:")
print(os.listdir(MODEL_DIR))


Files inside model directory:
['adapter_model.bin', 'tokenizer.model', 'tokenizer_config.json', 'tokenizer.json', 'special_tokens_map.json', 'README.md', 'adapter_config.json', 'training_args.bin']


In [2]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch, json

MODEL_DIR = "agileai_tinyllama_qlora"  # Your fine-tuned model

model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_DIR,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model.eval()
print("✨ Model loaded successfully!")


/apps/jupyter/6.5.4/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32000. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


✨ Model loaded successfully!


In [3]:
def build_inference_prompt(project_text: str) -> str:
    return (
        "You are AgileAI. Convert the given project description into exactly:\n"
        "- 1 Epic (E1)\n"
        "- At least 5 Features (F1, F2, ...)\n"
        "- For each Feature: 3–7 User Stories (US1, US2, ...)\n\n"
        "Use this strict JSON schema:\n"
        "{\n"
        "  \"epic\": {\"id\": \"E1\", \"title\": \"...\", \"description\": \"...\", \"acceptance_criteria\": \"...\"},\n"
        "  \"features\": [\n"
        "    {\n"
        "      \"id\": \"F1\",\n"
        "      \"title\": \"...\",\n"
        "      \"description\": \"...\",\n"
        "      \"acceptance_criteria\": \"...\",\n"
        "      \"user_stories\": [\n"
        "        {\"id\": \"US1\", \"title\": \"...\", \"description\": \"...\", \"acceptance_criteria\": \"...\"}\n"
        "      ]\n"
        "    }\n"
        "  ]\n"
        "}\n\n"
        "Rules:\n"
        "- Output **ONLY JSON**.\n"
        "- No extra text, no explanations.\n"
        "- No raw newlines inside strings (use \\n).\n\n"
        "### Project Description:\n"
        f"{project_text}\n\n"
        "### JSON Output:\n"
    )


In [4]:
def generate_raw_output(project_text, max_new_tokens=800, temperature=0.7, top_p=0.9):
    prompt = build_inference_prompt(project_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(output[0], skip_special_tokens=True)

    # Remove the prompt part → keep only model output
    return text[len(prompt):].strip()


In [5]:
def parse_json_safely(text: str):
    text = text.strip()
    start = text.find('{')
    end = text.rfind('}')

    if start == -1 or end == -1:
        print("❌ Could not detect JSON boundaries.")
        return None

    json_str = text[start:end+1]

    # Replace raw newlines inside strings
    json_str = json_str.replace("\r", " ").replace("\n", " ")

    try:
        data = json.loads(json_str)
        print("✅ JSON parsed successfully!")
        return data
    except Exception as e:
        print("❌ JSON parsing error:", e)
        print("Offending JSON snippet:\n", json_str[:500])
        return None


In [6]:
def generate_agile_output(project_text):
    raw = generate_raw_output(project_text)
    parsed = parse_json_safely(raw)
    return parsed, raw


In [9]:
test_file = "clean_text/AgroVision_clean.txt"   # change filename here

with open(test_file, "r", encoding="utf-8") as f:
    project_text = f.read()

agile_json, raw_output = generate_agile_output(project_text)

print("\n🔹 RAW OUTPUT (first 1000 chars):\n", raw_output[:1000])

print("\n🔹 PARSED JSON KEYS:")
if agile_json:
    print(list(agile_json.keys()))


✅ JSON parsed successfully!

🔹 RAW OUTPUT (first 1000 chars):
 {
  "project_id": "E1",
  "title": "AgroVision: Crop Disease Detection System",
  "description": "AgroVision introduces an automated convolutional neural network (CNN) based detection system capable of identifying common leaf based diseases with high accuracy, enabling timely intervention and improved agricultural outcomes.",
  "acceptance_criteria": [
    {
      "type": "disease",
      "description": "Disease classification based on images"
    }
  ],
  "user_stories": [
    {
      "id": "US1",
      "title": "Feature: Disease Classification",
      "description": "Implement a multi-classification CNN for disease classification",
      "acceptance_criteria": [
        {
          "type": "pass",
          "description": "Model accuracy meets or exceeds 90% threshold"
        }
      ]
    }
  ]
}

🔹 PARSED JSON KEYS:
['project_id', 'title', 'description', 'acceptance_criteria', 'user_stories']
